In [163]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [164]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [165]:
tf.keras.backend.clear_session()

In [166]:
subject = 'Acre - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [167]:
data = pd.read_csv('2003_mo_model_input_AC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.722906,0.691320,7.285707e+06,413409.255032,9.318417,6.587220e+06,3.260
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.718028,0.691617,7.294881e+06,413607.843560,9.319990,6.590338e+06,2.191
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.713149,0.691914,7.304055e+06,413806.432089,9.321563,6.593455e+06,2.154
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.708271,0.692212,7.313229e+06,414005.020618,9.323136,6.596572e+06,2.643
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.703393,0.692509,7.322403e+06,414203.609146,9.324709,6.599689e+06,2.546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.748
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.032
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.573
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.493


In [168]:
input_data = data.iloc[:split_index + 1,1:-1]
# mean = np.mean(input_data, axis=0)
# stddev =  np.std(input_data, axis=0)
# input_data = ((input_data - mean) /stddev)
scaler=MinMaxScaler()
scaled_data = scaler.fit_transform(input_data)
input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
0,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,0.056128,0.000000,0.000000,0.693537,0.455941,0.000000
1,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.054896,0.010205,0.006814,0.699191,0.464004,0.011752
2,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.053663,0.020410,0.013628,0.704845,0.472066,0.023504
3,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.052430,0.030616,0.020442,0.710499,0.480129,0.035256
4,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.051197,0.040821,0.027256,0.716153,0.488192,0.047008
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.906664,0.977176,0.966200,0.097050,0.000701,0.793320
188,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.904491,0.973916,0.962712,0.102440,0.000526,0.789444
189,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.902319,0.970655,0.959224,0.107830,0.000351,0.785568
190,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.900147,0.967395,0.955735,0.113220,0.000175,0.781692


In [169]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      4.105
1      3.898
2      4.209
3      2.994
4      3.720
       ...  
235      NaN
236      NaN
237      NaN
238      NaN
239      NaN
Name: Acre - Consumo de Cimento (t), Length: 240, dtype: float64

In [170]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
0,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,0.056128,0.000000,0.000000,0.693537,0.455941,0.000000
1,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.054896,0.010205,0.006814,0.699191,0.464004,0.011752
2,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.053663,0.020410,0.013628,0.704845,0.472066,0.023504
3,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.052430,0.030616,0.020442,0.710499,0.480129,0.035256
4,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.051197,0.040821,0.027256,0.716153,0.488192,0.047008
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.379453,0.024030,0.338208,0.039971,0.890665,0.956993,0.182507,0.995530,0.984319,0.992377,0.000007,0.085842,0.839039
158,0.332832,0.021531,0.371581,0.027548,0.893949,0.953662,0.168812,0.991059,0.983545,0.993070,0.000014,0.080835,0.838516
159,0.313954,0.019385,0.365349,0.037038,0.897232,0.950331,0.152959,0.986589,0.982772,0.993763,0.000021,0.075827,0.837993
160,0.294856,0.017671,0.404993,0.070841,0.900516,0.947000,0.137613,0.982119,0.981998,0.994456,0.000028,0.070819,0.837470


In [171]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       4.105
1       3.898
2       4.209
3       2.994
4       3.720
        ...  
157     9.081
158     9.545
159    10.130
160    14.451
161    13.218
Name: Acre - Consumo de Cimento (t), Length: 162, dtype: float64

In [172]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [173]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 13)

In [174]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
126,0.612440,0.052718,0.094523,0.201912,0.780557,0.994213,0.372054,0.731313,0.712498,0.910689,0.374302,0.655667,0.994431
127,0.626227,0.056653,0.200555,0.209545,0.784414,0.995177,0.375438,0.742775,0.713890,0.916766,0.367222,0.643610,0.995359
128,0.645442,0.060771,0.214993,0.238038,0.788271,0.996142,0.372254,0.754236,0.715281,0.922843,0.360142,0.631553,0.996287
129,0.673151,0.065105,0.053026,0.249843,0.792128,0.997106,0.369458,0.765698,0.716672,0.928919,0.353061,0.619495,0.997216
130,0.699949,0.069099,0.016760,0.266770,0.795984,0.998071,0.371547,0.777159,0.718063,0.934996,0.345981,0.607438,0.998144
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.906664,0.977176,0.966200,0.097050,0.000701,0.793320
188,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.904491,0.973916,0.962712,0.102440,0.000526,0.789444
189,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.902319,0.970655,0.959224,0.107830,0.000351,0.785568
190,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.900147,0.967395,0.955735,0.113220,0.000175,0.781692


In [175]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 13)

In [176]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [177]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
#     train, train_val = validation_splitter(train_input, 6)
#     target,target_val = validation_splitter(train_target, 6)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
#     optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(72, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(36, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(36, activation='tanh'),
        tf.keras.layers.Dense(18, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')    
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
#                         validation_data=(train_val,
#                                          target_val),
                        validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [178]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [179]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3222795219, 3789988419, 3705312079, 244898190, 353651419, 3854647553, 3411116183, 650079876, 1385670582, 2644573219]


Step: 0 ___________________________________________
val_loss: 2.9251530170440674
winner_seed: 3222795219


Step: 1 ___________________________________________
val_loss: 5.896622657775879


Step: 2 ___________________________________________
val_loss: 2.910875082015991
winner_seed: 3705312079


Step: 3 ___________________________________________
val_loss: 6.5798258781433105


Step: 4 ___________________________________________
val_loss: 6.532825946807861


Step: 5 ___________________________________________
val_loss: 6.437382698059082


Step: 6 ___________________________________________
val_loss: 6.3921074867248535


Step: 7 ___________________________________________
val_loss: 5.8000311851501465


Step: 8 ___________________________________________
val_loss: 6.634424686431885


Step: 9 ___________________________________________
val_loss: 6.597012042999268


final_se

In [180]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 117ms/step - loss: 203.0976 - val_loss: 122.7691
Epoch 2/10000
4/4 [==============================] - 0s 8ms/step - loss: 172.9166 - val_loss: 99.6488
Epoch 3/10000
4/4 [==============================] - 0s 7ms/step - loss: 150.9944 - val_loss: 82.5951
Epoch 4/10000
4/4 [==============================] - 0s 7ms/step - loss: 132.2609 - val_loss: 69.1174
Epoch 5/10000
4/4 [==============================] - 0s 8ms/step - loss: 116.8580 - val_loss: 59.0236
Epoch 6/10000
4/4 [==============================] - 0s 7ms/step - loss: 106.1309 - val_loss: 51.5094
Epoch 7/10000
4/4 [==============================] - 0s 7ms/step - loss: 96.0371 - val_loss: 46.1407
Epoch 8/10000
4/4 [==============================] - 0s 8ms/step - loss: 89.9072 - val_loss: 42.3662
Epoch 9/10000
4/4 [==============================] - 0s 7ms/step - loss: 84.4920 - val_loss: 39.6972
Epoch 10/10000
4/4 [==============================] - 0s 8ms/step - loss: 80.5779 

4/4 [==============================] - 0s 8ms/step - loss: 24.6202 - val_loss: 4.0658
Epoch 82/10000
4/4 [==============================] - 0s 7ms/step - loss: 24.3482 - val_loss: 3.9651
Epoch 83/10000
4/4 [==============================] - 0s 8ms/step - loss: 24.0669 - val_loss: 3.8714
Epoch 84/10000
4/4 [==============================] - 0s 7ms/step - loss: 23.7849 - val_loss: 3.7845
Epoch 85/10000
4/4 [==============================] - 0s 7ms/step - loss: 23.5239 - val_loss: 3.7023
Epoch 86/10000
4/4 [==============================] - 0s 7ms/step - loss: 23.2668 - val_loss: 3.6234
Epoch 87/10000
4/4 [==============================] - 0s 7ms/step - loss: 23.0315 - val_loss: 3.5466
Epoch 88/10000
4/4 [==============================] - 0s 7ms/step - loss: 22.7813 - val_loss: 3.4751
Epoch 89/10000
4/4 [==============================] - 0s 7ms/step - loss: 22.5479 - val_loss: 3.4068
Epoch 90/10000
4/4 [==============================] - 0s 7ms/step - loss: 22.3073 - val_loss: 3.3448
Epoch

4/4 [==============================] - 0s 8ms/step - loss: 15.4142 - val_loss: 4.3190
Epoch 162/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.3924 - val_loss: 4.3592
Epoch 163/10000
4/4 [==============================] - 0s 8ms/step - loss: 15.3724 - val_loss: 4.3973
Epoch 164/10000
4/4 [==============================] - 0s 8ms/step - loss: 15.3522 - val_loss: 4.4336
Epoch 165/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.3240 - val_loss: 4.4683
Epoch 166/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.3138 - val_loss: 4.5081
Epoch 167/10000
4/4 [==============================] - 0s 8ms/step - loss: 15.2911 - val_loss: 4.5474
Epoch 168/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.2682 - val_loss: 4.5808
Epoch 169/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.2535 - val_loss: 4.6182
Epoch 170/10000
4/4 [==============================] - 0s 8ms/step - loss: 15.2282 - val_loss: 4.6

4/4 [==============================] - 0s 7ms/step - loss: 14.8372 - val_loss: 6.3791
Epoch 242/10000
4/4 [==============================] - 0s 8ms/step - loss: 14.8385 - val_loss: 6.3937
Epoch 243/10000
4/4 [==============================] - 0s 8ms/step - loss: 14.8358 - val_loss: 6.4022
Epoch 244/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.8381 - val_loss: 6.4227
Epoch 245/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.8380 - val_loss: 6.4322
Epoch 246/10000
4/4 [==============================] - 0s 8ms/step - loss: 14.8321 - val_loss: 6.4434
Epoch 247/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.8330 - val_loss: 6.4551
Epoch 248/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.8349 - val_loss: 6.4698
Epoch 249/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.8278 - val_loss: 6.4834
Epoch 250/10000
4/4 [==============================] - 0s 8ms/step - loss: 14.8276 - val_loss: 6.4

4/4 [==============================] - 0s 8ms/step - loss: 14.8257 - val_loss: 6.7134
Epoch 322/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.8285 - val_loss: 6.7153
Epoch 323/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.8309 - val_loss: 6.7224
Epoch 324/10000
4/4 [==============================] - 0s 8ms/step - loss: 14.8306 - val_loss: 6.7204
Epoch 325/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.8290 - val_loss: 6.7195
Epoch 326/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.8323 - val_loss: 6.7249
Epoch 327/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.8261 - val_loss: 6.7285
Epoch 328/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.8277 - val_loss: 6.7372
Epoch 329/10000
4/4 [==============================] - 0s 8ms/step - loss: 14.8282 - val_loss: 6.7499
Epoch 330/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.8281 - val_loss: 6.7

...

Epoch 1041/10000
4/4 [==============================] - 0s 7ms/step - loss: 10.3314 - val_loss: 17.9810
Epoch 1042/10000
4/4 [==============================] - 0s 8ms/step - loss: 12.2378 - val_loss: 17.7582
Epoch 1043/10000
4/4 [==============================] - 0s 7ms/step - loss: 11.8467 - val_loss: 17.3700
Epoch 1044/10000
4/4 [==============================] - 0s 7ms/step - loss: 11.3047 - val_loss: 16.9006
Epoch 1045/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.4439 - val_loss: 16.4131
Epoch 1046/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.0190 - val_loss: 16.1984
Epoch 1047/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.3662 - val_loss: 16.3195
Epoch 1048/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.6872 - val_loss: 16.5733
Epoch 1049/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.2101 - val_loss: 16.8215
Epoch 1050/10000
4/4 [==============================] - 0s 7ms/step -

4/4 [==============================] - 0s 7ms/step - loss: 7.2510 - val_loss: 19.8332
Epoch 1121/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.1687 - val_loss: 19.9754
Epoch 1122/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.5615 - val_loss: 20.0863
Epoch 1123/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.9240 - val_loss: 20.1258
Epoch 1124/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.8620 - val_loss: 20.0068
Epoch 1125/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.3637 - val_loss: 19.7448
Epoch 1126/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.0693 - val_loss: 19.6158
Epoch 1127/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.2886 - val_loss: 19.5362
Epoch 1128/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.3900 - val_loss: 19.3830
Epoch 1129/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.7904 - val_l

4/4 [==============================] - 0s 7ms/step - loss: 7.6036 - val_loss: 21.7421
Epoch 1200/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.1697 - val_loss: 21.9686
Epoch 1201/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.0512 - val_loss: 22.2212
Epoch 1202/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.3417 - val_loss: 22.4338
Epoch 1203/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.7267 - val_loss: 22.4960
Epoch 1204/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.6796 - val_loss: 22.6075
Epoch 1205/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.1355 - val_loss: 22.6250
Epoch 1206/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.2248 - val_loss: 22.5158
Epoch 1207/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.3188 - val_loss: 22.2568
Epoch 1208/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.2603 - val_l

4/4 [==============================] - 0s 8ms/step - loss: 6.8219 - val_loss: 25.2593
Epoch 1279/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.8967 - val_loss: 25.3090
Epoch 1280/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.1033 - val_loss: 25.4169
Epoch 1281/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.7885 - val_loss: 25.5268
Epoch 1282/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.9154 - val_loss: 25.4993
Epoch 1283/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.0324 - val_loss: 25.3528
Epoch 1284/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.2239 - val_loss: 25.3356
Epoch 1285/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.7641 - val_loss: 25.3240
Epoch 1286/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.9525 - val_loss: 25.3911
Epoch 1287/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.0434 - val_l

4/4 [==============================] - 0s 7ms/step - loss: 6.7381 - val_loss: 25.0954
Epoch 1358/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.5564 - val_loss: 25.0568
Epoch 1359/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.2784 - val_loss: 25.1190
Epoch 1360/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.7014 - val_loss: 25.2189
Epoch 1361/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.1359 - val_loss: 25.2944
Epoch 1362/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.2958 - val_loss: 25.2216
Epoch 1363/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.2918 - val_loss: 25.1827
Epoch 1364/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.3640 - val_loss: 25.1295
Epoch 1365/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.0338 - val_loss: 25.1741
Epoch 1366/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.7322 - val_l

In [181]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [182]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 13ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,12.151371,12.152035,12.152174,12.152496,12.152602,12.152573,12.153013,12.154558,12.155586,12.156231,12.156905,12.15811,12.157083,12.158241,12.159542,12.159397,12.158718,12.157624,12.158833,12.160255,12.16068,12.161263,12.161792,12.16227,12.162741,12.163021,12.163061,12.162951,12.16295,12.162607
Target,13.552,15.826,14.869,14.734,12.754,11.925,13.114,8.92,9.724,9.737,10.136,14.403,12.871,15.749,14.269,15.367,14.595,9.811,11.188,10.381,8.08,11.154,12.508,12.126,14.496,16.723,15.253,16.531,15.206,10.16
Error,1.400629,3.673965,2.716826,2.581504,0.601398,0.227572,0.960987,3.234558,2.431586,2.41923,2.020905,2.24489,0.713918,3.590758,2.109458,3.207603,2.436282,2.346624,0.970833,1.779256,4.08068,1.007262,0.346209,0.036269,2.33326,4.559978,3.089939,4.36805,3.043051,2.002607


In [183]:
display(mae)
display(mape)

2.2178698

0.17591065

In [184]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [185]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[145.84766]] - Target[149.694]| =  Error: [[3.846344]]; MAPE:[[0.02569471]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[145.91571]] - Target[148.09900000000002]| =  Error: [[2.1832886]]; MAPE:[[0.01474209]]
1/1 [==============================] - 0s 13ms/step
Ano-5: |Prediction[[72.977325]] - Target[88.369]| =  Error: [[15.391678]]; MAPE:[[0.17417507]]


[array([[3.846344]], dtype=float32),
 array([[2.1832886]], dtype=float32),
 array([[15.391678]], dtype=float32)]

7.1404366

0.07153729